<a href="https://colab.research.google.com/github/wayne4918/Medical_ChatBot/blob/main/Medical_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving 전체통합.csv to 전체통합.csv


In [ ]:
import pandas as pd

df = pd.read_csv('전체통합.csv')

print(df)


                                                   intro  \
0      결핵은 흔히 폐에 병을 일으키지만 다른 장기나 조직에서도 발생할 수 있습니다. 폐결...   
1      결핵은 결핵균에 의해 감염되어 발생하는 전염성 질환으로, 감염 후 체내의 저항력이 ...   
2      결핵은 결핵균에 의해 발생하는 전염성 질환입니다. 주로 폐를 비롯한 신체기관에 침입...   
3      결핵은 호흡기 증상을 일으키는 주요한 감염성 질환입니다. 결핵에 감염된 사람은 기침...   
4      결핵은 전염성이 있는 질병이므로 감염 여부를 확인하기 위해서는 적절한 검사가 필요합...   
...                                                  ...   
22669  폐렴은 폐렴구균에 의한 감염으로 인해 발생하는 호흡기 감염 질환이며, 다양한 증상을...   
22670           폐렴은 폐 조직에 염증이 생기는 감염 질환으로, 다양한 증상을 보입니다.   
22671                    폐렴은 호흡기 감염으로 인해 발생하는 심각한 질환입니다.   
22672  폐렴은 폐렴구균이 폐 조직에 감염을 일으켜 발생하는 감염 질환으로, 다양한 증상을 ...   
22673  폐렴은 세균이나 바이러스 감염으로 인한 폐의 염증 질환으로, 기침, 가래, 호흡 곤...   

                                                    body  \
0      폐결핵의 초기 증상은 주로 발열, 체중 감소, 피로, 흉통 등이 있습니다. 이러한 ...   
1      폐결핵은 활동성 결핵 감염자로부터 나온 미세한 침방울에 감염되어 발생할 수 있습니다...   
2      결핵으로 인한 폐 증상은 흉통, 기침, 가래, 호흡곤란 등이 있습니다. 이러한 증상...   
3      결핵은 기침과 가래를 동반한 흔한 폐 질환입니다. 결핵으로

In [ ]:

df.head()


,intro,body,conclusion,disease_name_kor,department,disease_category
0,결핵은 흔히 폐에 병을 일으키지만 다른 장기나 조직에서도 발생할 수 있습니다. 폐결...,"폐결핵의 초기 증상은 주로 발열, 체중 감소, 피로, 흉통 등이 있습니다. 이러한 ...","결핵은 결핵균에 의해 유발되며, 이에 따라 다양한 증상이 나타날 수 있습니다. 진단...",결핵,내과,감염성질환
1,"결핵은 결핵균에 의해 감염되어 발생하는 전염성 질환으로, 감염 후 체내의 저항력이 ...",폐결핵은 활동성 결핵 감염자로부터 나온 미세한 침방울에 감염되어 발생할 수 있습니다...,결핵의 증상은 발병하는 부위와 감염 정도에 따라 다양하게 나타날 수 있습니다. 초기...,결핵,호흡기내과,감염성질환
2,결핵은 결핵균에 의해 발생하는 전염성 질환입니다. 주로 폐를 비롯한 신체기관에 침입...,"결핵으로 인한 폐 증상은 흉통, 기침, 가래, 호흡곤란 등이 있습니다. 이러한 증상...",결핵 증상이 발생하면 조기에 진단을 받아야 합니다. 결핵의 치료에는 2개월 이상의 ...,결핵,내과,감염성질환
3,결핵은 호흡기 증상을 일으키는 주요한 감염성 질환입니다. 결핵에 감염된 사람은 기침...,결핵은 기침과 가래를 동반한 흔한 폐 질환입니다. 결핵으로 인해 가래는 끈적하고 고...,결핵의 조기 발견과 치료가 중요합니다. 결핵 의심 증상이 있는 경우 의료진을 찾아가...,결핵,내과,감염성질환
4,결핵은 전염성이 있는 질병이므로 감염 여부를 확인하기 위해서는 적절한 검사가 필요합...,폐결핵의 가장 기본적인 진단은 객담 도말 검사입니다. 이 검사는 결핵균을 확인하기 ...,결핵 진단은 의심되는 증상이 나타날 경우 의사와 상담하여 정확한 검사와 진단을 받는...,결핵,내과,감염성질환


In [ ]:
file_path = '전체통합.csv'

df_full = pd.read_csv(file_path)

# 데이터프레임을 질병명별로 그룹화
grouped = df_full.groupby('disease_name_kor')

# 각 그룹에 대한 텍스트 데이터 결합
disease_texts = {}
for name, group in grouped:
    # intro, body, conclusion을 하나의 텍스트로 결합
    texts = group[['intro', 'body', 'conclusion']].apply(lambda x: ' '.join(x), axis=1)
    disease_texts[name] = texts.tolist()

# 예시: 결핵에 대한 텍스트 데이터
print(disease_texts['결핵'][:5])

['결핵은 흔히 폐에 병을 일으키지만 다른 장기나 조직에서도 발생할 수 있습니다. 폐결핵은 기침과 가래가 가장 흔한 증상입니다. 폐결핵의 초기 증상은 주로 발열, 체중 감소, 피로, 흉통 등이 있습니다. 이러한 증상이 나타나면 결핵 가능성이 높으니 주의해야 합니다. 기침, 객담, 발열, 체중 감소, 피로감, 호흡곤란 등의 증상이 있을 경우 의사를 방문하여 결핵에 대한 검사를 받아야 합니다. 결핵은 결핵균에 의해 유발되며, 이에 따라 다양한 증상이 나타날 수 있습니다. 진단은 주로 흉부 X선 촬영과 객담 검사를 통해 이루어집니다.', '결핵은 결핵균에 의해 감염되어 발생하는 전염성 질환으로, 감염 후 체내의 저항력이 약해지면 발병할 확률이 높아집니다. 결핵균은 주로 폐를 감염시키지만 신장, 신경, 뼈 등 다른 조직과 장기에도 침범할 수 있습니다. 폐결핵은 활동성 결핵 감염자로부터 나온 미세한 침방울에 감염되어 발생할 수 있습니다. 폐결핵의 증상은 주로 기침과 가래가 동반되며, 흉통이나 호흡곤란 등의 증상이 나타날 수 있습니다. 특히 활동성 결핵 감염자로부터 나온 미세한 침방울에 포함된 결핵균이 폐를 감염시킬 경우, 폐결핵의 증상이 더 악화될 수 있습니다. 결핵의 증상은 발병하는 부위와 감염 정도에 따라 다양하게 나타날 수 있습니다. 초기 증상으로는 피로, 체중감소, 기운이 없음 등이 나타날 수 있으며, 기침과 가래가 더 심하게 나타날 수 있습니다. 결핵은 조기에 진단하고 적절한 치료를 받는 것이 중요합니다.', '결핵은 결핵균에 의해 발생하는 전염성 질환입니다. 주로 폐를 비롯한 신체기관에 침입하여 결핵을 유발할 수 있습니다. 결핵은 폐, 신장, 신경, 뼈 등 다양한 조직에서 발병할 수 있습니다. 결핵은 결핵균으로 인해 발생하는 다양한 증상을 가지고 있습니다. 결핵으로 인한 폐 증상은 흉통, 기침, 가래, 호흡곤란 등이 있습니다. 이러한 증상은 일반적으로 진행 중인 결핵을 나타낼 수 있으며, 치료가 필요한 정도로 진행될 수 있습니다. 폐 증상이 있는 경우 결핵을